In [146]:
import pulp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
distance_matrix = pd.read_csv('dataset/Distance_Matrix.csv').drop('Unnamed: 0', axis=1)
biomass_history = pd.read_csv('dataset/Biomass_History.csv')


### Detalhes:

- *Variáveis de Decisão*: As variáveis `x`, `y`, `depot`, e `refinery` representam as decisões de alocação e seleção.
- *Função Objetivo*: A função objetivo combina o custo de transporte e a incompatibilidade da previsão de biomassa.
- *Restrições*: As restrições garantem que cada local de colheita seja atribuído a um depósito, cada depósito a uma refinaria, e outras restrições específicas do problema.

Este código é um esboço geral e pode precisar de ajustes e detalhamento adicionais com base nos dados e requisitos específicos do problema.

In [147]:
year = '2011'
B = biomass_history[year].to_numpy()
D = distance_matrix.to_numpy()
C = 20_000

num_warehouses = 25
num_harvesting_sites = 2418

In [148]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417
0,0.0000,11.3769,20.4557,38.1227,45.3810,54.9915,78.6108,118.6750,102.6639,113.4309,...,683.8771,687.6310,697.3246,669.3962,667.6788,665.5775,662.0291,665.9655,673.2073,681.4235
1,11.3769,0.0000,9.0788,28.9141,36.1724,45.7829,69.4022,78.2329,93.4553,111.1832,...,681.6295,685.3833,695.0769,667.1485,665.4311,663.3298,659.7815,663.7178,670.9596,679.1758
2,20.4557,9.0788,0.0000,22.3791,29.6374,39.2478,62.8671,71.6979,86.9203,111.7859,...,682.2323,685.9861,695.6796,667.7513,666.0339,663.9326,660.3843,664.3206,671.5623,679.7786
3,38.1227,28.9141,22.3791,0.0000,11.8343,23.5413,41.8396,50.6703,65.8927,82.5852,...,681.4226,685.1765,694.8701,666.9417,665.2243,663.1230,659.5746,663.5110,670.7528,678.9690
4,45.3810,36.1724,29.6374,11.8343,0.0000,11.7070,24.3986,33.2293,53.9901,65.1442,...,663.9816,667.7355,677.4291,649.5007,647.7833,645.6820,642.1336,646.0700,653.3118,661.5280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,671.2005,668.9528,669.5556,669.9084,652.1102,640.4032,627.7116,622.9663,638.9532,617.7221,...,109.0007,112.7545,122.4481,12.3091,9.1558,0.0000,14.5629,22.3860,36.6284,44.8446
2414,663.4455,661.1978,661.8006,662.1534,644.3551,632.6481,619.9566,615.2112,631.1981,609.9671,...,101.2456,104.9995,114.6931,26.4955,23.3422,14.1864,0.0000,14.5984,28.8407,37.0570
2415,663.7748,661.5271,662.1298,662.4826,644.6844,632.9774,620.2859,615.5405,631.5274,610.2964,...,84.1469,87.9008,97.5943,34.3098,31.1565,22.0007,12.8105,0.0000,14.2423,22.4586
2416,671.0165,668.7688,669.3715,669.7244,651.9261,640.2191,627.5276,622.7822,638.7691,617.5381,...,91.3886,95.1425,104.8361,48.5521,45.3988,36.2430,27.0528,14.2423,0.0000,12.4741


In [149]:
def print_matrix(matrix):
    print(pd.DataFrame(matrix).to_string(index=True, header=True))

print_matrix(D[:5, :5])

         0        1        2        3        4
0   0.0000  11.3769  20.4557  38.1227  45.3810
1  11.3769   0.0000   9.0788  28.9141  36.1724
2  20.4557   9.0788   0.0000  22.3791  29.6374
3  38.1227  28.9141  22.3791   0.0000  11.8343
4  45.3810  36.1724  29.6374  11.8343   0.0000


In [150]:
import pulp

warehouse_locations = [i for i in range(num_possible_warehouses_locations)]  # Creates a list of all warehouses
# Creates a list of all demand nodes
harvest_sites = [i for i in range(num_harvesting_sites)]

cost_matrix = D * B

In [151]:
# The cost data is made into a dictionary
costs = pulp.makeDict([harvest_sites, warehouse_locations], cost_matrix, 0)

In [152]:
warehouses = pulp.LpVariable.dicts("W%s", warehouse_locations, cat=pulp.LpBinary)
serv_harvest=pulp.LpVariable.dicts("H%s_W%s", (harvest_sites, warehouse_locations), cat=pulp.LpBinary)

In [205]:
# Creates the 'prob' variable to contain the problem data
prob = pulp.LpProblem("Supply_Chain_Optimization", pulp.LpMinimize)

cost_of_transportation = pulp.lpSum([serv_harvest[h][w] * costs[h][w] for w in warehouse_locations for h in harvest_sites])
cost_of_underutilization = pulp.lpSum([C - pulp.lpSum([serv_harvest[h][w] for h in harvest_sites]) for w in warehouse_locations])
a= 0.001
b=1
prob += a * cost_of_transportation + b * cost_of_underutilization

In [206]:
# Constraints

# Constraint 1: Number of warehouses must be less or equal to 5
prob += (
    pulp.lpSum([warehouses[w] for w in warehouse_locations]) <= num_warehouses, 
    "Maximum_number_of_warehouses"
    )

# Constraint 2: Each harvesting site must be assigned to exactly one warehouse
for h in harvest_sites:
    prob += (
        pulp.lpSum([serv_harvest[h][w] for w in warehouse_locations]) == 1,
        f"Harvesting_site_{h}_must_be_assigned_to_exactly_one_warehouse"
    )

# Constraint 3: If a harvesting site is assigned to a warehouse, the warehouse must be selected
for h in harvest_sites:
    for w in warehouse_locations:
        prob += (
            serv_harvest[h][w] <= warehouses[w],
            f"Harvesting_site_{h}_must_be_assigned_to_warehouse_{w}_only_if_warehouse_{w}_is_selected"
        )

# Constraint 4: Limit the capacity of each warehouse
for w in warehouse_locations:
    prob += (
        pulp.lpSum([serv_harvest[h][w] * B[h] for h in harvest_sites]) <= C,
        f"Limit_the_capacity_of_warehouse_{w}"
    )


In [207]:
prob.solve()
# print the status of the solution
print("Status:", pulp.LpStatus[prob.status])

# print the value of the objective
print("Total Cost of Transportation = ", pulp.value(prob.objective))

# print the value of the variables
for v in prob.variables():
    print(v.name, "=", v.varValue)


In [181]:

# create a dataframe with the solution
solution = pd.DataFrame(columns=['Harvesting Site', 'Warehouse', 'Selected'])
for v in prob.variables():
    if v.varValue == 1 and v.name.startswith('H'):
        splitted = v.name.split('_')
        solution = pd.concat(
            [solution, pd.DataFrame([[splitted[0][1:], splitted[1][1:], v.varValue]], columns=['Harvesting Site', 'Warehouse', 'Selected'])]
            )
#solution = solution[solution['Selected'] == 1].reset_index(drop=True)
solution = solution.astype(int)
solution


,Harvesting Site,Warehouse,Selected
0,0,0,1
0,1000,13,1
0,1001,13,1
0,1002,13,1
0,1003,13,1
...,...,...,...
0,997,13,1
0,998,13,1
0,999,13,1
0,99,13,1


In [203]:
solution['HS_Longitude']= solution['Harvesting Site'].apply(lambda idx: biomass_history['Longitude'][int(idx)])
solution['HS_Latitude']= solution['Harvesting Site'].apply(lambda idx: biomass_history['Latitude'][int(idx)])
solution['W_Longitude']= solution['Warehouse'].apply(lambda idx: biomass_history['Longitude'][int(idx)])
solution['W_Latitude']= solution['Warehouse'].apply(lambda idx: biomass_history['Latitude'][int(idx)])
solution

,Harvesting Site,Warehouse,Selected,HS_Longitude,HS_Latitude,W_Longitude,W_Latitude
0,0,0,1,71.33144,24.66818,71.33144,24.66818
0,1000,13,1,72.04807,22.81437,72.36657,24.66818
0,1001,13,1,72.12769,22.81437,72.36657,24.66818
0,1002,13,1,72.20732,22.81437,72.36657,24.66818
0,1003,13,1,72.28694,22.81437,72.36657,24.66818
...,...,...,...,...,...,...,...
0,997,13,1,71.80919,22.81437,72.36657,24.66818
0,998,13,1,71.88882,22.81437,72.36657,24.66818
0,999,13,1,71.96844,22.81437,72.36657,24.66818
0,99,13,1,71.88882,24.26518,72.36657,24.66818


In [204]:
solution.Warehouse.value_counts()

13    2392
14       3
0        1
22       1
8        1
7        1
6        1
5        1
4        1
3        1
2        1
24       1
23       1
20       1
21       1
1        1
19       1
18       1
17       1
16       1
15       1
12       1
11       1
10       1
9        1
Name: Warehouse, dtype: int64